## 4.3 Derrube pra fora

#### Objetivo:
implemente o regularizador dropout na rede neural feita em Python puro.

**Comentário**: algumas tarefas vão apresentar palavras e conceitos que ainda não
vimos em sala. Parte do desafio é justamente se informar sobre estes conceitos.

## Introdução

Neste notebook, exploramos a técnica de Dropout, um método simples e eficiente de regularização para redes neurais que ajuda a evitar o sobreajuste. A ideia central do Dropout é, a cada minibatch de treinamento, “desligar” aleatoriamente uma fração dos neurônios — ou seja, zerar suas ativações — de modo que o modelo não dependa demais de nenhuma conexão específica. Com isso, cada atualização de peso é feita em uma rede levemente diferente, o que força o sistema a aprender representações mais robustas e distribuídas.

Diferente da implementação convencional de dropout, foi feito uma implementação onde, em cada neurônio, existe uma chance dele ser ser desligado com base em um valor sorteado e a taxa de dropout, a cada inferência da rede e não a cada época, como o convencional. Essa implementação é uma forma mais simples, que pode com funcionar com datasets mais robustos, porém, para fins didáticos, ele se mostrou eficiente. Ao longo do notebook você verá a implementação dessa técnica e seu uso em um conjunto de dados simples fornecido. 


In [2]:
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np 

C:\Users\marco24038\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
import math

class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo
        self.grad = 0

    def __repr__(self):
        return f"Valor(data={self.data})"
    
    def __add__(self, outro_valor):
        """Realiza a operação: self + outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data + outro_valor.data
        operador_mae = "+"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_adicao():
            self.grad += resultado.grad
            outro_valor.grad += resultado.grad
            
        resultado.propagar = propagar_adicao
        
        return resultado
    
    def __mul__(self, outro_valor):
        """Realiza a operação: self * outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data * outro_valor.data
        operador_mae = "*"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_multiplicacao():
            self.grad += resultado.grad * outro_valor.data # grad_filho * derivada filho em relação a mãe
            outro_valor.grad += resultado.grad * self.data
            
        resultado.propagar = propagar_multiplicacao
        
        return resultado
    
    def exp(self):
        """Realiza a operação: exp(self)"""
        progenitor = (self, )
        data = np.exp(self.data)
        operador_mae = "exp"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_exp():
            self.grad += resultado.grad * data 
        
        resultado.propagar = propagar_exp
        
        return resultado
    
    def __pow__(self, expoente):
        """Realiza a operação: self ** expoente"""
        assert isinstance(expoente, (int, float))
        progenitor = (self, )
        data = self.data ** expoente
        operador_mae = f"**{expoente}"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_pow():
            self.grad += resultado.grad * (expoente * self.data ** (expoente - 1))
        
        resultado.propagar = propagar_pow
        
        return resultado
    
    def __truediv__(self, outro_valor):
        """Realiza a operação: self / outro_valor"""
        return self * outro_valor ** (-1)
    
    def __neg__(self):
        """Realiza a operação: -self"""
        return self * -1
    
    def __sub__(self, outro_valor):
        """Realiza a operação: self - outro_valor"""
        return self + (-outro_valor)
    
    def __radd__(self, outro_valor):
        """Realiza a operação: outro_valor + self"""
        return self + outro_valor
    
    def __rmul__(self, outro_valor):
        """Realiza a operação: outro_valor * self"""
        return self * outro_valor
    
    def sig(self):
        """Realiza a operação: exp(self) / (exp(self) + 1)"""
        e_neg = (-self).exp()
        return Valor(1) / (Valor(1)+ e_neg)
    
    def propagar(self):
        pass
    
    def propagar_tudo(self):
        
        self.grad = 1
        
        ordem_topologica = []
        
        visitados = set()

        def constroi_ordem_topologica(v):
            if v not in visitados:
                visitados.add(v)
                for progenitor in v.progenitor:
                    constroi_ordem_topologica(progenitor)
                ordem_topologica.append(v)

        constroi_ordem_topologica(self)
        
        for vertice in reversed(ordem_topologica):
            vertice.propagar()

In [4]:
import random

class Neuronio:
    def __init__(self, num_dados_entrada):
        self.vies = Valor(random.uniform(-1, 1))
        
        self.pesos = []
        for i in range(num_dados_entrada):
            self.pesos.append(Valor(random.uniform(-1, 1)))
            
    def __call__(self, x, dropout):
        
        assert len(x) == len(self.pesos)
        
        prob_desl = random.uniform(0,1)
        soma = 0
        for info_entrada, peso_interno in zip(x, self.pesos):
            soma += info_entrada * peso_interno
            
        soma += self.vies  
        
        if prob_desl < dropout:
            dado_de_saida = Valor(0)
        else:
            dado_de_saida = soma.sig()
        
        return dado_de_saida       
    
    def parametros(self):
        return self.pesos + [self.vies]

In [5]:
class Camada:
    def __init__(self, num_neuronios, num_dados_entrada):
        neuronios = []
        
        for _ in range(num_neuronios):
            neuronio = Neuronio(num_dados_entrada)
            neuronios.append(neuronio)
            
        self.neuronios = neuronios     
        
    def __call__(self, x, dropout):
        dados_de_saida = []
        
        for neuronio in self.neuronios:
            informacao = neuronio(x,dropout)
            dados_de_saida.append(informacao)
            
        if len(dados_de_saida) == 1:
            return dados_de_saida[0]
        else:        
            return dados_de_saida  
    
    def parametros(self):
        params = []
        
        for neuronio in self.neuronios:
            params_neuronio = neuronio.parametros()
            params.extend(params_neuronio)
        
        return params

In [6]:
class MLP:
    def __init__(self, num_dados_entrada, num_neuronios_por_camada):
        
        percurso = [num_dados_entrada] + num_neuronios_por_camada
        
        camadas = []
        
        for i in range(len(num_neuronios_por_camada)):
            camada = Camada(num_neuronios_por_camada[i], percurso[i])
            camadas.append(camada)
            
        self.camadas = camadas
        
    def __call__(self, x, dropout):
        for camada in self.camadas[:-1]:
            x = camada(x, dropout)

        x = self.camadas[-1](x, 0)
        return x
    
    def parametros(self):
        params = []
        
        for camada in self.camadas:
            parametros_camada = camada.parametros()
            params.extend(parametros_camada)
            
        return params

In [47]:
x = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
x = np.array(x)
y_true = [1, 0, 0.2, 0.5]

NUM_DADOS_DE_ENTRADA = 3  
NUM_DADOS_DE_SAIDA = 1    
CAMADAS_OCULTAS = [3, 2]  

arquitetura_da_rede = CAMADAS_OCULTAS + [NUM_DADOS_DE_SAIDA]

minha_mlp = MLP(NUM_DADOS_DE_ENTRADA, arquitetura_da_rede)

In [54]:
NUM_EPOCAS = 1000
TAXA_DE_APRENDIZADO = 0.3
dropout = 0.5
perdas= []
for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = []
    for exemplo in x:
        previsao = minha_mlp(exemplo, dropout)
        y_pred.append(previsao)

    # loss
    erros = []
    for yt, yp in zip(y_true, y_pred):
        residuo = yp - yt
        erro_quadratico = residuo ** 2
        erros.append(erro_quadratico)        
    loss = sum(erros)
    perdas.append(loss.data)

    # zero grad
    for p in minha_mlp.parametros():
        p.grad = 0

    # backpropagation
    loss.propagar_tudo()

    # atualiza parâmetros
    for p in minha_mlp.parametros():
        p.data = p.data - p.grad * TAXA_DE_APRENDIZADO
    

    # mostra resultado (opcional)
    print(epoca, loss.data)

0 0.5501176444991599
1 0.49724960167584653
2 0.6331005122512335
3 0.5461158203343135
4 0.5175593895396227
5 0.6001098733543347
6 0.48985315245934463
7 0.6363955212205267
8 0.4934900544809984
9 0.5435961842163615
10 0.5314558839616841
11 0.5657924543235209
12 0.5889556211806886
13 0.475353025666116
14 0.6149756593829233
15 0.5815184068186094
16 0.6444424858663009
17 0.6054134441622522
18 0.5141150475336062
19 0.5208433674003952
20 0.5298664072968073
21 0.547246054425405
22 0.6013621403853249
23 0.537730837742047
24 0.5870999411475217
25 0.5537360077490319
26 0.5187292673360815
27 0.62418084130333
28 0.6536867695284221
29 0.5725162801205558
30 0.5819277500711386
31 0.5869104196085084
32 0.5211335369618375
33 0.4966302301605426
34 0.5506307105032289
35 0.5424114080967135
36 0.4705954185810515
37 0.6514730289849274
38 0.5567557163483398
39 0.6395139166458442
40 0.5667202209202429
41 0.5317220486198504
42 0.5375028539078456
43 0.6293408776879144
44 0.5470100804957773
45 0.603603349725757
46

In [55]:
print(y_pred)

[Valor(data=0.5624706257839246), Valor(data=0.2296621853849746), Valor(data=0.2506467744188815), Valor(data=0.5503107606301684)]


## Conclusão

Nessa notebook, é possível identifcar que com o dropout ativo, ocorre uma variação da taxa de perda da rede neural. Isso acontece, porque com a variação de neurônios devido ao dropout, visto que eles são desligados aleatóriamente a cada época de acordo com a taxa de dropout, faz com que neurônios com viéses mais influentes sejam desligados da rede, o que causa uma oscilação da funçao de perda, entretanto isso serve para evitar que o neurônio realize overfitting e "decore" dados. Vale lembrar que essa não é uma implementação ideal do dropout, porém possibilita a visualização dessa variação, o que é suficiente para o contexto acadêmico dessa atividade. 

## Referências

- Cassar, Daniel R. - Material da disciplina de Redes Neurais e Algoritmos genéticos. 2025
- [Medium](https://medium.com/@mangeshsalunke1309/dropout-regularization-technique-a770fbcd9692)